# <font color='#eb3483'> Transforming Pandas DataFrames and Series </font>

Where pandas really shines is in it's ability to transform data - allowing you to tidy up a dataset in just a few lines of code. In this notebook, we'll be exploring this functionality. Remember, most of a data scientist's time is spent cleaning and transforming data!

In this notebook we will cover:

1. Removing rows and columns
1. Mathematical operations
1. Naming operations
1. Aggregations

We're going to work with the AirBnB dataset again, but only a few columns. When we load a dataframe from the csv file, we can specify the columns we want to use with `usecols`.

In [ ]:
import pandas as pd
columns = ["room_id", "host_id", "room_type", "neighborhood", "reviews", "overall_satisfaction","accommodates", "bedrooms", "price"]
df = pd.read_csv("data/airbnb.csv", usecols=columns, index_col="room_id").sort_index()
df.head()

## <font color='#eb3483'> 1. Removing Rows and Columns </font>
To remove rows and columns, we can use dataframe's `.drop` method. By default `.drop` removes rows based on the index value (**not** row position).

Drop has two important arguments:
* `index/columns`: Specify the index values and/or column names that you want to drop.
* `inplace`: With this argument, you can chose if you want to transform the original dataframe or if you want the drop function to return a copy of the transformed dataframe. The default value is ```False```.

To find out more on the ```.drop``` method and its arguments, type ```?df.drop```.

In [ ]:
df.drop(index = [6499, 17031]) # drops the rows where room_id is 6499 or 17031 (this is the index)

In [ ]:
?df.drop

In [ ]:
df.drop(columns=["reviews", "price"]) # drops two columns

You can drop rows and columns at the same time. e.g.

```python
df.drop(index = [6499, 17031], columns=["reviews", "price"])
```

Columns can also be dropped using Python's ```del``` function:
```python
del df['reviews']
```
Note that this is an "inplace" operation, so it modifies the original dataframe, but does not return anything. It can only be applied to one column at a time and only when the column name is provided inside square brackets.

This will work:
```python
del df['reviews'], df['price']
```

These will **not** work:
```python
del df[['reviews','price']]
del df.reviews
```

## <font color='#eb3483'> 2. Mathematical Operations </font>

Very often we want to manipulate a column to get something that makes more sense. Maybe getting a value per hour, or price per week or turning a birthdate into an age etc.

### <font color='#eb3483'>  Multiplication </font>

You can either use the `*` or the `.multiply()` method to multiply two columns or multiply columns by a number.

For example, we can calculate the weekly price for the listings:

In [ ]:
df["price_per_week"] = df.price * 7 # or df["price_per_week"] = df.price.multiply(7)
df.head()

### <font color='#eb3483'>   Division </font>
We can use either `/` or the `.divide()` to divide.

For example, we can calculate the number of people per bedroom:

In [ ]:
df["people_per_bedroom"] = df.accommodates / df.bedrooms # or df.accommodates.divide(df.bedrooms)
df.head(10)

<font color='#eb3483'> Exercise: </font> Try adding or subtracting columns.

## <font color='#eb3483'> 3. Naming Operations </font>

We can change the name of the columns by changing the column names list `df.columns`. For example, we can rename the columns and make them capitalized.

In [ ]:
new_column_names = df.columns.str.title() # converts the current column names to title case
df.columns = new_column_names
df.head()

Alternatively, we can use the ```.rename``` method to rename either the row indices (with ```index=...```) or column names (with ```columns=...```). In both cases, we provide a dictionary that maps from the old name (dict key) to the new name (dict value). For example:

In [ ]:
df.rename(columns = {"Host_Id": "My_Funky_New_Name"}) # could use inplace=True to modify original

### <font color='#eb3483'>  Replace </font>

`.replace` allows us to replace values in the data (not the indices). For example, if we want to change the numerical values in the ```Overall_Satisfaction``` column to an ordinal scale we can do it by passing a dictionary to ```.replace``` in much the same way as we did for ```.rename```:

By default  `replace` replaces the values in all the columns. We should rather replace values only in the column we need.

In [ ]:
df1 = df.Overall_Satisfaction.replace(
    {
        5:"Best",
        4: "Good",
        3: "OK",
        2: "Not so great",
        1: "Worst",
        0: "No Information"
    })

In [ ]:
df1

## <font color='#eb3483'> 4. Aggregations </font>
The goal of aggregations is to allow us to get an aggregated view of sub-sections of our data. Before we begin, let's read in our data again to erase all the edits that we may have made above:

In [ ]:
df = pd.read_csv("data/airbnb.csv", usecols=columns)
df.head()

### <font color='#eb3483'>  Groupby </font>

`groupby` allows us to group the dataframe based on its features.

More precisely, Pandas' ```groupby``` enables a process called [split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/groupby.html).
* **split**: Separates the dataframe based on the specified groups
* **apply**: Applies a function to each one of the groups
* **combine**: Combines the results into a new dataframe

For example, if we want to know how many listings there are in each neighborhood. This means that for every ```neighborhood``` (the "group"), we want to count up the unique values of ```room_id```.

We start by grouping the data:

In [ ]:
df.groupby("host_id")

This returns a `DataFrameGroupBy ` object, which is a special dataframe object that separates the dataframe by group.

In [ ]:
df.nunique() # gives you the number of unique values in each column

In [ ]:
df.groupby("neighborhood").nunique() # gives you the number of unique values in each column BY group

In [ ]:
# just for the "room_id" column:
df.groupby("neighborhood").room_id.nunique()

By default, the columns we use to group become the index, if we want them to stay as columns we can use the argument `as_index = False`.

In [ ]:
df.groupby("neighborhood", as_index=False).room_id.nunique()

<font color='#eb3483'> Exercise: </font> Calculate the average listing price by room type and host. **Hint:** Use the ```.mean()``` method to get the average (in place of ```.nunique()``` above).

In [ ]:
df.groupby(["room_type","host_id"], as_index=False).price.mean()

We can apply any function to a grouped dataframe and pandas will pass the function on to the underlying split dataframes under the hood:

In [ ]:
df.groupby(["neighborhood"]).price.describe()

### <font color='#eb3483'>  Aggregate </font>

We have seen how to apply in-built methods like ```.sum``` and ```.mean``` to groups in a dataframe. What if we want to apply some other funky functions to each group, possibly ones that we have written ourselves? The pandas `aggregate` method allows us to do just that!

In [ ]:
def funky(x):
    return (min(x)+max(x))/2 # this returns a single number

df.groupby(["neighborhood"]).price.aggregate(funky)

In [ ]:
df.groupby(["neighborhood"]).price.aggregate([min, funky, max]) # multiple functions all at once!

The `aggregate` function even let's us specify what columns we want to apply each aggregation function to using a dictionary:

In [ ]:
df.groupby(["neighborhood"]).aggregate({'price': [funky, max], 'accommodates':min})

### <font color='#eb3483'> Transform </font>

The `transform` function allows us to apply a function to the grouped data without aggregation. For example, what if we wanted to center the price of each listing to see it's price relative to the average price in each neighbourhood?

In [ ]:
def normalize(x):     # x is a single column in the dataframe
    return x-x.mean() # this returns a series of the same length as x

df['norm_price'] = df.groupby(["neighborhood"])['price'].transform(normalize)
df.head()

### <font color='#eb3483'> Apply </font>

The ```apply``` function extends the functionality of `aggregate` and `transform` by allowing you to apply a function to **multiple** columns from the split up dataframes. It can return a single value for each group (like `aggregrate`) or a transformed series (like `transform`). It is the best of both worlds!

Let's say we wanted to get the average price per person accommodated for each neighbourhood? That is, we get a **single number** for each neighborhood...

In [ ]:
def AvePricePerAccom(x):                            # x is a dataframe for a specific group
    return (x['price']/x['accommodates']).mean()    # this returns a single value

df.groupby(["neighborhood"]).apply(AvePricePerAccom)

Below we compute the normalized price per person for each neighborhood. The function returns a **series** for each neighborhood...

In [ ]:
def NormPricePerAccom(x):                                           # x is a dataframe for a specific group
    return (x['price']-x['price'].mean())/x['accommodates']         # this returns a series

df.groupby(["neighborhood"]).apply(NormPricePerAccom)

### <font color='#eb3483'>  Crosstab </font>

`pd.crosstab` allows us to cross tabulate two columns in our dataset and returns the number/proportion of observations that fall into each cell in the resulting table.

In [ ]:
pd.crosstab(df.neighborhood, df.room_type)

We can use the argument `normalize` to get percentages instead of totals:
- `normalize="all"` returns total percentages (% of the total dataframe)
- `normalize="index"` returns percentages per row
- `normalize="columns"` returns percentages per column

In [ ]:
pd.crosstab(df.neighborhood, df.room_type, normalize="all").head()

Now we can see the percentage of listings per neighborhood broken down by room type:

In [ ]:
pd.crosstab(df.neighborhood, df.room_type, normalize="index").head()

And we can see how many of each room type are on each neighborhood:

In [ ]:
pd.crosstab(df.neighborhood, df.room_type, normalize="columns").head()

### <font color='#eb3483'>  Pivot Table </font>

The `.pivot_table` method performs the same function as pivot tables in Excel. It turns rows into columns based on the values on the columns (that is, it "pivots" the data).

This function has different arguments:

- `index`: the columns whose values should become rows
- `columns`: the columns whose values should become columns
- `values`: the columns we want to aggregate
- `aggfunc`: the aggregate function applied to the values (mean by default)

For example, if we want to calculate the average satisfaction by ```room_type``` for each ```neighborhood```:

In [ ]:
df = pd.read_csv("data/airbnb.csv", usecols=columns, index_col="room_id").sort_index()
df.pivot_table(index="neighborhood",
               columns='room_type',
               values='overall_satisfaction',
               aggfunc='mean').head()

<hr>

# <font color='#eb3483'> LET'S PRACTICE! </font>

# <font color='#eb3483'> Transforming Pandas DataFrames and Series  </font>

Work on these excercises for 10 mins (or for homework depending on how well we do with time)

For these exercises we are going to use a new dataset, the 2016 US Primary elections (`primary_results.csv` in our data folder). Start by importing pandas and reading in our data:

The dataset has the following columns:

- *state*
- *state_abbreviation*
- *county*
- *fips* county identifier
- *party*
- *candidate*
- *votes* votes the candidate got in the county
- *fraction_votes* percentage of the total county votes the candidate got

For each problem - think about how you would work this out first. Talk yourself through each step (or even jot it down) and then code it.

### <font color='#eb3483'> Exercise 1 </font>
Overall, which percentage of votes did every party get?

### <font color='#eb3483'> Exercise 2 </font>

Who is the democrat candidate that got the most votes in manhattan? and in the state of New York?

### <font color='#eb3483'> Exercise 3 </font>
How many votes did Donald Trump receive in Texas?

### <font color='#eb3483'> Exercise 4 </font>

Let's consider democrat states those where the democrats got more votes and republican states those where the republican candidates got more votes. Which states are democrat and which republican?


*hint: one way to find out is by doing a pivot table using the sum as an aggregating function*

### <font color='#eb3483'> Exercise 5 </font>

In how many of the republican states was Donald Trump the most voted republican candidate?